## DDQN in Hockey Enviroment - Evaluation

Evaluate a trained DDQN against various environments

Base DQN implementation adapted from HW7

In [1]:
import os
import sys
from importlib import reload

import numpy as np
from tqdm.notebook import tqdm

# Adding the parent directory to the path to enable importing
root_dir = os.path.dirname(os.path.abspath("../"))
if root_dir not in sys.path:
    sys.path.append(root_dir)

import DDQN.DDQN as ddqn
from DDQN.DDQN import DuelingDQNAgent
from DDQN.evaluation import compare_agents, display_stats
import hockey.hockey_env as h_env

reload(h_env)
reload(ddqn)

/home/kivanc/miniconda3/envs/rl/lib/python3.11/site-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment Hockey-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/kivanc/miniconda3/envs/rl/lib/python3.11/site-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment Hockey-One-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


<module 'DDQN.DDQN' from '/home/kivanc/ders/RL/project/RL-Hockey/DDQN/DDQN.py'>

In [2]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

## Environment & Agent Initialization

In [3]:
env = h_env.HockeyEnv(mode=h_env.Mode.NORMAL)
env.reset()

(array([-3.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  3.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.57698822,  0.72862673,  0.        ,
         0.        ,  0.        ,  0.        ]),
 {'winner': 0,
  'reward_closeness_to_puck': 0.0,
  'reward_touch_puck': 0.0,
  'reward_puck_direction': 0.0})

In [4]:
STATE_PATH = "../models/hockey_mixed_dev/"

agent_player = DuelingDQNAgent(
    env.observation_space,
    env.discrete_action_space,
    hidden_sizes=[256, 256],
)
agent_player.load_state(STATE_PATH)

agent_opp_weak = h_env.BasicOpponent(weak=True)
agent_opp_strong = h_env.BasicOpponent(weak=False)

## Evaluation

### Winning Rates Against Opponents & Match Statistics

In [5]:
weak_opp_stats = compare_agents(
    agent_player, agent_opp_weak, env, num_matches=1000, tqdm=tqdm
)
strong_opp_stats = compare_agents(
    agent_player, agent_opp_strong, env, num_matches=1000, tqdm=tqdm
)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [6]:
print("#" * 40)
print("Stats Against Weak Opponent:")
display_stats(weak_opp_stats)

print("#" * 40)
print("Stats Against Strong Opponent:")
display_stats(strong_opp_stats)

########################################
Stats Against Weak Opponent:
Player Observation Mean:
  x pos player one: -2.0440269768503874
  y pos player one: 0.004576087458980476
  angle player one: -0.009205523992135832
  x vel player one: 0.4126231353362218
  y vel player one: -0.05830181658017967
  angular vel player one: 0.008531989861695077
  x player two: 2.675923206910976
  y player two: 0.017140769690138367
  angle player two: -0.006100770133496
  y vel player two: 0.009538306898026094
  y vel player two: 0.0019467434421993387
  angular vel player two: -0.0003301826760893594
  x pos puck: -0.1152260440873716
  y pos puck: -0.009109355148699982
  x vel puck: -0.9357098459935692
  y vel puck: 0.006792591175240485
  left player puck keep time: 0.8244170547483092
  right player puck keep time: 0.8190031101383929

Relative Std. Change in Agent Observations:
  x pos player one: 0.0480672074684908
  y pos player one: 0.5682092812960798
  angle player one: 0.7653761394465192
  x vel playe

### Rendered Demonstration Against Strong Opponent

In [7]:
obs_buffer = []
reward_buffer = []
obs, _ = env.reset()
obs_opp = env.obs_agent_two()

done = False
trunc = False
step = 0
while not (done or trunc):
    step += 1
    env.render()

    a1_discr = agent_player.act(obs)
    a1 = env.discrete_to_continous_action(a1_discr)
    a2 = agent_opp_strong.act(obs_opp)

    obs, r, done, trunc, _ = env.step(np.hstack([a1, a2]))
    obs_buffer.append(obs)
    reward_buffer.append(r)

    obs_opp = env.obs_agent_two()

    if done or trunc:
        print(f"Episode done in {step} steps")
        break

Episode done in 206 steps


In [8]:
env.close()